In [22]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import time
import pickle

## Following are visulization environment setup referred from TA: Boxi Xia

In [23]:
def setView(ctr,camera_pos=(1,1,1), lookat=(0, 0, 0), up=(0.5, 0.5, 1)):
    """
    set the view given a view control handel ctr
    """
    ctr.set_constant_z_far(100) # camera z far clip plane
    ctr.set_constant_z_near(0.01)# camera z near clip plane
    
def customVisualization(geomtry_list):
    """
    helper function to create a visualization given a list of o3d geometry
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    for g in geomtry_list:
        vis.add_geometry(g)
    ctr = vis.get_view_control()
    setView(ctr)
    vis.run()
    vis.destroy_window() # close the window when finished
    
def createPlane(r=10, dr=0.1):
    """
    return a plane located at (0,0,0),and with plane normal = (0,0,1)
    r: radius of the plane
    dr:discretization radius of the grid
    """
    bounds = np.array([[-r, -r, 0],[r, r, 0]])/dr
    bounds = np.stack((np.floor(bounds[0]), np.ceil(bounds[1])))*dr
    nx, ny, nz = np.ceil((bounds[1]-bounds[0])/dr).astype(int)
#     print(nx,ny)
    xyz = np.reshape([[[[i, j, 0], [i+1, j, 0], [i, j+1, 0],
                       [i, j+1, 0], [i+1, j, 0], [i+1, j+1, 0]] for i in range(nx-1)] for j in range(ny-1)], (-1, 3))
    xyz = (xyz - ((nx-1)/2,(ny-1)/2,0))*dr
#     xyz, bounds, (nx, ny, nz) = create_grid(bounds, dr)
#     print(nx, ny, nz)
    triangles = np.arange(xyz.shape[0]).reshape((-1,3))
    plane = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(
        xyz), o3d.utility.Vector3iVector(triangles))
    # assign checkerboard color pattern
    c0 = (0.729, 0.78, 0.655) # first color
    c1 = (0.533, 0.62, 0.506) # second color
    colors = np.reshape([[np.tile(c0 if (i+j)%2 else c1,(6,1)) for i in range(nx-1)] for j in range(ny-1)],(-1,3))
    plane.vertex_colors = o3d.utility.Vector3dVector(colors)
    plane.compute_triangle_normals()
    return plane

# create a plane
plane = createPlane()
# create a coordinate frame
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])


## Following are the functions to generate masses and springs of a six-arm  robot

In [24]:
def generateRandomArmLength(min_length = 0, max_length = 8, num_arm = 6):
    #generate a 6 elements arm length array
    np.random.seed()
    arm_length = [np.random.randint(min_length, max_length) for i in range(num_arm)]
    arm_length.insert(0, 0)
    return np.array(arm_length, dtype = np.int64)
    
def createRobotLocation(arm_length, edge, center_point = (0,0,0)):
    '''
    ROBOT_HEIGHT is the initial height we added to the robot, arm_length is the number of cubes per arm
    In this robot, we have one body with 8 cubes, 6 arms with 4 cubes each. You can set the arm_length to other number
    The center point of the robot is index14， mass_pos_list[14] = center_point
    '''
    mass_pos_list = []
    x0, y0, z0 = center_point
    #create the location of center body
    body = [[x0 + i * edge,y0 + j * edge,z0 + k * edge] for i in range(-1,2) for j in range(-1,2) for k in range(-1,2)]
    mass_pos_list.extend(body)
    #create the location of eight arms
    arm_vector = [[2*edge, 0, 0], [-2*edge, 0,0], [0, 2*edge, 0], [0,-2*edge,0], [0, 0, 2*edge], [0,0,-2*edge]]
    for vector in arm_vector:
        if vector[0] > 0:
            arm = [[2*edge + i*edge, 0.5 * j * np.sqrt(2*(edge**2)), z0+0.5 * k * np.sqrt(2*(edge**2))] \
                   for i in range(int(arm_length[1])) for j in [-1,0,1] for k in [-1,0,1] if (j!=k) and (j*k==0)]
        elif vector[0] < 0:
            arm = [[-2*edge - i*edge, 0.5 * j * np.sqrt(2*(edge**2)), z0+0.5 * k * np.sqrt(2*(edge**2))] \
                   for i in range(int(arm_length[2])) for j in [-1,0,1] for k in [-1,0,1] if (j!=k) and (j*k==0)]
        elif vector[1] > 0:
            arm = [[0.5 * i * np.sqrt(2*(edge**2)), 2*edge+j*edge, z0+0.5 * k * np.sqrt(2*(edge**2))] \
                   for j in range(int(arm_length[3])) for i in [-1,0,1] for k in [-1,0,1] if (i!=k) and (i*k==0)]
        elif vector[1] < 0:
            arm = [[0.5 * i * np.sqrt(2*(edge**2)), -2*edge-j*edge, z0+0.5 * k * np.sqrt(2*(edge**2))] \
                   for j in range(int(arm_length[4])) for i in [-1,0,1] for k in [-1,0,1] if (i!=k) and (i*k==0)]
        elif vector[2] > 0:
            arm = [[0.5 * i * np.sqrt(2*(edge**2)), 0.5 * j * np.sqrt(2*(edge**2)), z0+2*edge+k*edge,] \
                   for k in range(int(arm_length[5])) for i in [-1,0,1] for j in [-1,0,1] if (i!=j) and (i*j==0)]
        elif vector[2] < 0:
            arm = [[0.5 * i * np.sqrt(2*(edge**2)), 0.5 * j * np.sqrt(2*(edge**2)), z0-2*edge-k*edge,] \
                   for k in range(int(arm_length[6])) for i in [-1,0,1] for j in [-1,0,1] if (i!=j) and (i*j==0)]
        mass_pos_list.extend(arm)
    return np.array(mass_pos_list, dtype = np.float64)

def rotateRobotOnGround(mass_pos_list, max_robot_height):
    #rotate the robot in x,y,z axis
    rotation_x = np.radians(45)
    true_height = max_robot_height
    ground_height = np.sqrt(2) * true_height / 2
    rotation_y = np.arctan2(ground_height, true_height)
    rotation_z = np.radians(0)

    c_x, s_x = np.cos(rotation_x), np.sin(rotation_x)
    R_x = np.array(((1,0,0),(0,c_x,-s_x),(0,s_x,c_x)), dtype = np.float64)
    c_y, s_y = np.cos(rotation_y), np.sin(rotation_y)
    R_y = np.array(((c_y,0,s_y),(0,1,0),(-s_y,0,c_y)), dtype = np.float64)
    c_z, s_z = np.cos(rotation_z), np.sin(rotation_z)
    R_z = np.array(((c_z,-s_z,0),(s_z,c_z,0),(0,0,1)), dtype = np.float64)

    temp1 = (R_x @ mass_pos_list.T).T
    temp2 = (R_y @ temp1.T).T
    temp3 = (R_z @ temp2.T).T
    rotated_mass_pos_list = temp3
    
    #find the lowest max, set its position on ground while other masses will move too
    lowest_mass_z_pos = np.min(rotated_mass_pos_list[:,2]) 
    rotated_mass_pos_list[:,2] -= (lowest_mass_z_pos - 0.2)
    
    center_point = [0,0,0]
    cx, cy, cz = rotated_mass_pos_list[14]
    center_point = [cx,cy,cz]
    return rotated_mass_pos_list, center_point

def getMassDistance(m1_pos, m2_pos):
    return np.linalg.norm(np.array(m1_pos) - np.array(m2_pos), ord = 2)   

def getSpringList(mass_pos_list, long_diagonal, spring_k):
    spring_list = []
    mass_num = len(mass_pos_list)
    for i in range(mass_num):
        for j in range(i+1, mass_num):
            dis = getMassDistance(mass_pos_list[i], mass_pos_list[j])
            if dis <= long_diagonal +0.001:
                spring_list.append([spring_k, dis, i,j])
    return spring_list

In [25]:
#get all of the properties in numpy array of mass and spring
def getMassSpringProperties(arm_length, edge, long_diagonal, spring_k):
    mass_pos_list = np.asarray(createRobotLocation(arm_length, edge))
    max_robot_height = np.max(arm_length) #the max arm length decide how height the robot is
    mass_pos_list, center_point = np.asarray(rotateRobotOnGround(mass_pos_list, max_robot_height))
    spring_list = np.asarray(getSpringList(mass_pos_list, long_diagonal, spring_k))

    num_mass = len(mass_pos_list)
    #print(f"The number of mass is {num_mass}")
    mass_m = np.ones(num_mass,dtype=np.float64) * 0.1
    mass_pos = np.asarray(mass_pos_list,dtype=np.float64)
    mass_vel = np.zeros((num_mass,3),dtype=np.float64)
    mass_acc = np.zeros((num_mass,3),dtype=np.float64)

    num_spring =len(spring_list)
    #print(f"The number of spring is {num_spring}")
    spring_k = np.asarray(spring_list[:,0],dtype=np.float64)
    spring_rest = np.asarray(spring_list[:,1],dtype=np.float64) # rest length
    spring_ind = np.asarray(spring_list[:,2:],dtype=np.int64)
    return num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, spring_rest, spring_ind, center_point

In [26]:
#the robot has 6 arms, each arm has its own set of k, b, c. To make the representation easy, we split the spring in 7 groups
#, which are 1 body group and 6 arm groups
def splitSpring(spring_ind, num_mass, num_spring, arm_length):
    body_spring_ind = []
    arm0_spring_ind = []
    arm1_spring_ind = []
    arm2_spring_ind = []
    arm3_spring_ind = []
    arm4_spring_ind = []
    arm5_spring_ind = []
    num_mass_body = 27
    num_arm = 6
    
    arm_index_range = []
    right_index = num_mass_body
    for length in arm_length[1:]:
        left_index = right_index
        right_index = left_index + length * 4
        arm_index_range.append((left_index, right_index))
    arm_index_range = np.asarray(arm_index_range, dtype = np.int64)
    
    #assign each spring_ind's index into a 2d array
    for index, each in enumerate(spring_ind):
        mass1_index, mass2_index = each
        if mass1_index in range(arm_index_range[0][0], arm_index_range[0][1]) and mass2_index in range(arm_index_range[0][0], arm_index_range[0][1]):
            arm0_spring_ind.append(index)
        elif mass1_index in range(arm_index_range[1][0], arm_index_range[1][1]) and mass2_index in range(arm_index_range[1][0], arm_index_range[1][1]):
            arm1_spring_ind.append(index)
        elif mass1_index in range(arm_index_range[2][0], arm_index_range[2][1]) and mass2_index in range(arm_index_range[2][0], arm_index_range[3][1]):
            arm2_spring_ind.append(index)
        elif mass1_index in range(arm_index_range[3][0], arm_index_range[3][1]) and mass2_index in range(arm_index_range[3][0], arm_index_range[3][1]):
            arm3_spring_ind.append(index)
        elif mass1_index in range(arm_index_range[4][0], arm_index_range[4][1]) and mass2_index in range(arm_index_range[4][0], arm_index_range[4][1]):
            arm4_spring_ind.append(index)
        elif mass1_index in range(arm_index_range[5][0], arm_index_range[5][1]) and mass2_index in range(arm_index_range[5][0], arm_index_range[5][1]):
            arm5_spring_ind.append(index)
        else:
            body_spring_ind.append(index)
    return [body_spring_ind, arm0_spring_ind, arm1_spring_ind, arm2_spring_ind, arm3_spring_ind, arm4_spring_ind, arm5_spring_ind]

In [27]:
#get the velocity of robot
def getFitnessVelocity(center_mass_pos, T_MAX, center_point):
    center_x, center_y, _ = center_point
    cur_x, cur_y, _ = center_mass_pos
    #print(f"center point in getFitnessVelocity fun: {center_mass_pos}, {center_point}")
    diff_x, diff_y = center_x - cur_x, center_y - cur_y
    dis = np.linalg.norm([diff_x, diff_y], ord = 2)
    velocity = dis / T_MAX
    return velocity

In [28]:
#get the maximum diameter per cycle
def getRobotMaxSizePerCycle(velocity, omega, robot_length):
    cycle = 2 * np.pi / omega  # second per cycle
    velocity_per_cycle = cycle * velocity #meter per cycle
    max_size_per_cycle = velocity_per_cycle / robot_length #how many robot length the robot move per cycle
    return max_size_per_cycle

# Change Here! 
## The randomParameters function helps you to self-identify the properties range and make your own properties

In [29]:
def randomParameters(num_arm = 6):
    np.random.seed()
    parameters = np.zeros((7,5), dtype = object)
    
    omega_arm_range = [3,12]
    arm_length = generateRandomArmLength(min_length = 1, max_length = 8, num_arm = 6)
    #print(f"the arm_length is {arm_length}")
    k_body_range = [3000,5000]
    k_arm_range = [7000,20000]
    b_arm_range = [-0.4, 0.4]
    
    #in the order of arm_length, omega, k, b, c
    for i in range(num_arm+1):
        if i == 0:
            parameters[i,0] = arm_length[i]
            parameters[i,1] = 0
            parameters[i,2] = np.random.randint(k_body_range[0], k_body_range[1])
            parameters[i,3] = 0
            parameters[i,4] = 0
        else:
            parameters[i,0] = arm_length[i]  #len(arm_length) = 6 while num_arm+1 = 7
            parameters[i,1] = np.random.randint(omega_arm_range[0], omega_arm_range[1])
            parameters[i,2] = np.random.randint(k_arm_range[0], k_arm_range[1])
            parameters[i,3] = np.random.uniform(b_arm_range[0], b_arm_range[1])
            parameters[i,4] = np.random.uniform(-2 * np.pi / parameters[i,1], 2 * np.pi / parameters[i,1])
    return parameters    #in the order of arm_length, omega, k, b, c

In [30]:
# This function run the simulation in a visual GUI
def runSimulation(plane, lsd, parameters, num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, \
                  spring_rest, spring_ind, center_point, different_arm_spring_index, DT, T_MAX, OMEGA):
    GROUND_K = 100000
    DAMPING = 0.999
    #EDGE = 0.1 
    #SHORT_DIAGONAL = np.sqrt(2)/10
    #LONG_DIAGONAL = np.sqrt(3)/10
    #CENTER_POINT = (0,0,0)
    #RADIUS = 0.01
    DT = DT
    OMEGA = parameters[:,1]
    GRAVITY = 9.81
    MU_S = 1.0 #static friction coefficient of rubber on concrete
    MU_K = 0.8 #sliding friction coefficient of rubber on concrete
    T = 0
    T_MAX = T_MAX
    
    ended=False
    def signalEnd(vis):
        nonlocal ended
        ended =True
        
    vis = o3d.visualization.VisualizerWithKeyCallback()
    # ref:https://www.glfw.org/docs/3.3/group__keys.html
    # press key Q or ESC to close the window
    vis.register_key_callback(81, signalEnd)# key Q 
    vis.register_key_callback(256, signalEnd)# key escape
    vis.create_window()
    
    vis.add_geometry(lsd)
    vis.add_geometry(plane)   
    
    # view control
    ctr = vis.get_view_control()
    setView(ctr, camera_pos=(1,1,1), lookat=(0, 0, 0), up=(0,0, 1))
    
    loop_num = 0
    #change the spring_k:
    spring_current_k = np.copy(spring_k)
    for i in range(7):
        spring_current_k[different_arm_spring_index[i]] = parameters[i,2]
    
    while T < T_MAX:
        T += DT
        #set the force to zero every loop
        spring_force = np.zeros((num_spring),dtype=np.float64)
        mass_force = np.zeros((num_mass,3),dtype=np.float64)
        

        #change the spring_rest with t
        spring_current_rest = np.copy(spring_rest)

        for i in range(7):
            if i != 0:
                spring_current_rest[different_arm_spring_index[i]] *= 1 + parameters[i,3] * np.sin(OMEGA[i] * T + parameters[i,4])
                
        #get the spring_force
        spring_vec = mass_pos[spring_ind[:,1]] - mass_pos[spring_ind[:,0]]
        spring_vec_norm = np.linalg.norm(spring_vec,axis = 1)
        spring_unit_vec = spring_vec/ spring_vec_norm[:, np.newaxis]
        spring_force = ((spring_vec_norm - spring_current_rest) * spring_current_k)[:,np.newaxis] * spring_unit_vec
        np.add.at(mass_force, spring_ind[:,0], spring_force)
        np.add.at(mass_force, spring_ind[:,1], -spring_force)

        #get the gravity force
        mass_force[:,2] -= mass_m * GRAVITY

        #get the restoration_force
        mass_pos_below_zero = mass_pos[:,2] <= 0 #give a bool array when the z value of mass_pos is below zero

        #get coulomb friction
        normal_force_below_zero = mass_pos[:,2] < 0

        friction_available_index = np.asarray([i for i, val in enumerate(mass_pos_below_zero & normal_force_below_zero) if val])
        if np.any(friction_available_index):
            horizontal_force = np.sqrt(mass_force[friction_available_index,0]**2 + mass_force[friction_available_index,1]**2)
            normal_force = mass_force[friction_available_index,2]
            static_friction_index = np.array([i for i, val in zip(friction_available_index, horizontal_force < (-normal_force * MU_S)) \
                                                if val], dtype = np.int64)
            slide_friction_index = np.array([i for i, val in zip(friction_available_index, horizontal_force >= (-normal_force * MU_S)) \
                                               if val], dtype = np.int64)
            mass_force[static_friction_index,0] = 0.0
            mass_force[static_friction_index,1] = 0.0
            mass_vel[static_friction_index,0] = 0.0
            mass_vel[static_friction_index,1] = 0.0

            for index in slide_friction_index:
                if mass_force[index,0] < 0:
                    mass_force[index,0] += np.abs(mass_force[index,2]) * MU_K
                elif mass_force[index,0] >= 0:
                    mass_force[index,0] -= np.abs(mass_force[index,2]) * MU_K  
                elif mass_force[index,1] < 0:
                    mass_force[index,1] += np.abs(mass_force[index,2]) * MU_K    
                elif mass_force[index,1] >= 0:
                    mass_force[index,1] -= np.abs(mass_force[index,2]) * MU_K    
                    
        if np.any(mass_pos_below_zero):
            mass_force[mass_pos_below_zero, 2] -= GROUND_K * mass_pos[mass_pos_below_zero,2]

        #update the mass position
        mass_acc = mass_force/mass_m[:,np.newaxis]
        mass_vel += mass_acc * DT
        mass_vel *= DAMPING
        mass_pos += mass_vel * DT
        
        if loop_num % 100 == 0:
            lsd.points = o3d.utility.Vector3dVector(mass_pos)
            #o3d.visualization.draw_geometries([lsd])
            vis.update_geometry(lsd)
            vis.poll_events()
            vis.update_renderer()
            
        loop_num += 1
    return mass_pos, center_point

In [31]:
#This function run the simulation one time with one parameter set.
def runOneGeneration(parameters, num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, spring_rest, \
                     spring_ind, different_arm_spring_index, DT, T_MAX, OMEGA):
    SPRING_K = 10000
    GROUND_K = 100000
    DAMPING = 0.999
    DT = DT
    OMEGA = parameters[:,1]
    GRAVITY = 9.81
    MU_S = 1.0 #static friction coefficient of rubber on concrete
    MU_K = 0.8 #sliding friction coefficient of rubber on concrete
    T = 0
    T_MAX = T_MAX
    
    #change the spring_k:
    spring_current_k = np.copy(spring_k)
    for i in range(7):
        spring_current_k[different_arm_spring_index[i]] = parameters[i,2]
   
    #while loop
    loop_num = 0
    
    #used to calculate the kenimatic energy and potential energy
    total_energy_list = []
    stable_status = False
    center_point = [0,0,0]

    #total_energy = 0
    
    while T < T_MAX:
        T += DT
        #ke = 0
        #pe = 0

        #set the force to zero every loop
        spring_force = np.zeros((num_spring),dtype=np.float64)
        mass_force = np.zeros((num_mass,3),dtype=np.float64)
        
        #change the spring_rest with t
        spring_current_rest = np.copy(spring_rest)

        for i in range(7):
            if i != 0:
                spring_current_rest[different_arm_spring_index[i]] *= 1 + parameters[i,3] * np.sin(OMEGA[i] * T + parameters[i,4])

        #get the spring_force
        spring_vec = mass_pos[spring_ind[:,1]] - mass_pos[spring_ind[:,0]]
        spring_vec_norm = np.linalg.norm(spring_vec,axis = 1)
        spring_unit_vec = spring_vec/ spring_vec_norm[:, np.newaxis]
        spring_force = ((spring_vec_norm - spring_current_rest) * spring_current_k)[:,np.newaxis] * spring_unit_vec
        np.add.at(mass_force, spring_ind[:,0], spring_force)
        np.add.at(mass_force, spring_ind[:,1], -spring_force)
        #mass_force[spring_ind[:,0]] += spring_force[:,np.newaxis] * spring_unit_vec
        #mass_force[spring_ind[:,1]] -= spring_force[:,np.newaxis] * spring_unit_vec
        #pe += np.sum(0.5 * spring_current_k * np.power(spring_vec_norm - spring_current_rest, 2))

        #get the gravity force
        mass_force[:,2] -= mass_m * GRAVITY
        #pe += np.sum(GRAVITY * mass_m * mass_pos[:,2])

        #get the restoration_force
        mass_pos_below_zero = mass_pos[:,2] <= 0 #give a bool array when the z value of mass_pos is below zero

        #get coulomb friction
        normal_force_below_zero = mass_pos[:,2] < 0
        
        friction_available_index = np.asarray([i for i, val in enumerate(mass_pos_below_zero & normal_force_below_zero) if val])
        if np.any(friction_available_index):
            horizontal_force = np.sqrt(mass_force[friction_available_index,0]**2 + mass_force[friction_available_index,1]**2)
            normal_force = mass_force[friction_available_index,2]
            static_friction_index = np.array([i for i, val in zip(friction_available_index, horizontal_force < (-normal_force * MU_S)) \
                                                if val], dtype = np.int64)
            slide_friction_index = np.array([i for i, val in zip(friction_available_index, horizontal_force >= (-normal_force * MU_S)) \
                                               if val], dtype = np.int64)
            mass_force[static_friction_index,0] = 0.0
            mass_force[static_friction_index,1] = 0.0
            mass_vel[static_friction_index,0] = 0.0
            mass_vel[static_friction_index,1] = 0.0

            for index in slide_friction_index:
                if mass_force[index,0] < 0:
                    mass_force[index,0] += np.abs(mass_force[index,2]) * MU_K
                elif mass_force[index,0] >= 0:
                    mass_force[index,0] -= np.abs(mass_force[index,2]) * MU_K  
                elif mass_force[index,1] < 0:
                    mass_force[index,1] += np.abs(mass_force[index,2]) * MU_K    
                elif mass_force[index,1] >= 0:
                    mass_force[index,1] -= np.abs(mass_force[index,2]) * MU_K    
                    
        if np.any(mass_pos_below_zero):
            mass_force[mass_pos_below_zero, 2] -= GROUND_K * mass_pos[mass_pos_below_zero,2]
            #pe += np.sum(0.5 * GROUND_K * np.power(mass_pos[mass_pos_below_zero,2],2))

        #update the mass position
        mass_acc = mass_force/mass_m[:,np.newaxis]
        mass_vel += mass_acc * DT
        #ke += np.sum(0.5 * mass_m * np.power(np.linalg.norm(mass_vel, axis = 1), 2))
        mass_vel *= DAMPING
        mass_pos += mass_vel * DT
        
        #let the robot get stable then calculate the velocity, so here get the center_point when robot is stable
        if stable_status == False and T > 1:
            stable_status = True
            cx, cy, cz = mass_pos[14]
            center_point = [cx,cy,cz]
            
        #total_energy_list.append(pe+ke)
        loop_num += 1

    return mass_pos, center_point, total_energy_list

In [32]:
#get the initial parent_list and generate children_list from parent_list
def getInitialParent(parent_size):
    parent_list = np.asarray([randomParameters(num_arm = 6) for i in range(parent_size)])
    return parent_list
def getChildren(parent_list, mutation_rate = 0.08, crossover_rate = 0.2):
    num_parent = len(parent_list)
    num_mutation = int(np.ceil(mutation_rate * num_parent * 7 * 5)) #parameters shape is (7,3)
    num_crossover = int(np.ceil(crossover_rate * num_parent * 7))
    if num_crossover % 2 != 0:
        num_crossover += 1
    #print(f"The num of crossover is {num_crossover}")
    
    #mutation
    #print(f"The parent_list is {parent_list}")
    flatten_parent_list = parent_list.flatten()
    #print(f"The flatten_parent_list is {flatten_parent_list}")
    random_mutation_index = (np.random.choice(np.arange(num_parent * 7 * 5), size = num_mutation, replace = False))
    #print(f"The mutation_index is {random_mutation_index}")
    for index in random_mutation_index:
        if index % 5 == 0:
            if index % 35 == 0:
                continue
            random_num = np.random.randint(-2,3)
            flatten_parent_list[index] += random_num
            if flatten_parent_list[index] < 0:
                flatten_parent_list[index] = 0
        elif index % 5 == 1:
            if index % 35 == 1:
                continue
            random_num = np.random.randint(-2,3)
            flatten_parent_list[index] += random_num
            if flatten_parent_list[index] < 3:
                flatten_parent_list[index] = 3
        elif index % 5 == 2:
            random_num = np.random.randint(-1000,1000)
            flatten_parent_list[index] += random_num
            if flatten_parent_list[index] < 5000:
                flatten_parent_list[index] = 5000
        elif index % 35 == 3 or index % 35 == 4:
            continue
        else:
            random_scale = np.random.uniform(0.8,1.2)
            flatten_parent_list[index] *= random_scale
    mutated_children_list = np.reshape(flatten_parent_list, (num_parent,7,5))
    #print(f"The mutated_children_list is {mutated_children_list}")
    
    #crossover
    vstack_children_list = np.vstack(mutated_children_list)
    #print(f"The vstack_children_list is {vstack_children_list}")
    random_crossover_index = (np.random.choice(np.arange(num_parent * 7), size = num_crossover))
    #print(f"The crossover_index is {random_crossover_index}")
    for i in np.arange(0,len(random_crossover_index),2):
        #print(f"i is {i}")
        if random_crossover_index[i] % 7 == 0 or random_crossover_index[i+1] %7 == 0:
            continue
            #print(f"Can't crossover between body and arms")
        else:
            crossover_index = np.random.choice(np.arange(5), size = np.random.randint(0,6), replace = False) #random choose parameter to switch
            for index in crossover_index:
                temp = vstack_children_list[random_crossover_index[i]][index]
                vstack_children_list[random_crossover_index[i]][index] = vstack_children_list[random_crossover_index[i+1]][index]
                vstack_children_list[random_crossover_index[i+1]][index] = temp
    crossovered_children_list = np.reshape(vstack_children_list, (num_parent,7,5))
    #print(f"The crossovered_children_list is {crossovered_children_list}")
    return crossovered_children_list

In [33]:
def selection(velocity_list, parent_children_list, parent_size):
    #print(f"The velocity_list is {velocity_list}")
    #print(f"The parent_children_list is {parent_children_list}")
     #get the new velocity_list and parent_list relatively
    add_index_velocity_list = [[i, velocity_list[i]] for i in range(len(velocity_list))]
    sort_velocity_list = sorted(add_index_velocity_list, key = lambda x: x[1], reverse = True)
    sort_velocity_index_list = np.array(sort_velocity_list)[:,0].astype(np.int64)  #get the velocity index in descending velocity

    array_velocity_list = np.array(velocity_list)
    sort_array_velocity_list = array_velocity_list[sort_velocity_index_list]  #get the sorted velocity descending
    sort_parent_children_list = parent_children_list[sort_velocity_index_list]  #get the sorted parent_children list in descending velocity
    
    total_fitness = np.sum(sort_array_velocity_list)
    distance_between_pointers = total_fitness / parent_size
    start_pointer = np.random.uniform(0, distance_between_pointers)
    pointers = [start_pointer + i * distance_between_pointers for i in range(parent_size)]
    
    keep = [] #get the index
    for p in pointers:
        i = 0
        while np.sum(sort_array_velocity_list[:i+1]) < p:
            i += 1
        keep.append(i)
    
    velocity_list = sort_array_velocity_list[keep].tolist()
    print(f"The new velocity_list is {velocity_list}")
    parent_list = sort_parent_children_list[keep]
    #print(f"The new parent_list is {parent_list}")
    
    return velocity_list, parent_list

# Run Here!
## Get the parameters of parent_list, velocity per generation

In [35]:
def evolutionary(generation = 10, parent_size = 25, previous_parent_list = None, previous_best_velocity_list = None):
    
    start_time = time.process_time()
    #important initial properties
    WEIGHT = 0.1
    ARM_LENGTH = 4 #change the arm length here
    ROBOT_HEIGHT = 0.1 * ARM_LENGTH + 0.1 + 0.04 * (ARM_LENGTH-1) #just above the ground
    ROBOT_LENGTH =  0.1 * ARM_LENGTH * 2 + 0.2 #the longest distance between all pairs of masses
    SPRING_K = 10000
    GROUND_K = 100000
    DAMPING = 0.999
    EDGE = 0.1 
    SHORT_DIAGONAL = np.sqrt(2)/10
    LONG_DIAGONAL = np.sqrt(3)/10
    CENTER_POINT = (0,0,0)
    RADIUS = 0.01
    DT = 0.0001
    OMEGA = 9 #change the omega here
    GRAVITY = 9.81
    MU_S = 1.0 #static friction coefficient of rubber on concrete
    MU_K = 0.8 #sliding friction coefficient of rubber on concrete
    T = 0
    T_MAX = 7
    
    
    #start from beginning
    if np.any(previous_parent_list) == None or np.any(previous_best_velocity_list) == None:
        velocity_list = []  #[velocity]
        best_velocity_list = []  #only install the first parent's velocity
        parent_list = getInitialParent(parent_size) #This is initial parent list
    else:
        velocity_list = []
        best_velocity_list = previous_best_velocity_list
        parent_list = previous_parent_list

    for i in range(generation):
        #showing the progress 
        print(f"Generation: {i}")

        best_velocity = 0 #for each generation, record the best velocity

        #get the children and stack parent and children together for comparation
        children_list = getChildren(parent_list, mutation_rate = 0.05, crossover_rate = 0.4) #change the mutation_rate and crossover_rate here
        parent_children_list = np.vstack([parent_list, children_list])

        #If it is the first generation,go over the all parent_list and children_list to get the velocties
        if i == 0: 
            for i in range(len(parent_children_list)):
                print(f"Parent_Children_#{i}")
                parameters = parent_children_list[i]
                num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, spring_rest, spring_ind, center_point\
                    = getMassSpringProperties(arm_length = parameters[:,0], edge = EDGE, long_diagonal = LONG_DIAGONAL, spring_k = SPRING_K)
                different_arm_spring_index = splitSpring(spring_ind, num_mass, num_spring, arm_length = parameters[:,0])
                #print(f"The arm-length is {parameters[:,0]}")
                print(f"The parameters are {parameters}")

                mass_pos, center_point, total_energy_list = \
                    runOneGeneration(parameters, num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, spring_rest, spring_ind, different_arm_spring_index, DT, T_MAX, OMEGA)     
                velocity = getFitnessVelocity(mass_pos[14], T_MAX, center_point)
                print(f"the velocity at parent_children {i} is !!!{velocity}!!!")
                
                '''
                #visulize the initial robot shape, close the window to continue
                lsd = o3d.geometry.LineSet()
                lsd.points = o3d.utility.Vector3dVector(mass_pos)
                lsd.lines = o3d.utility.Vector2iVector(spring_ind)
                plane = createPlane()
                #o3d.visualization.draw_geometries([lsd])
                customVisualization([lsd, plane])
                '''

                velocity_list.append(velocity)
                '''
                if velocity > 2:
                    print('WARNING！！！！！！')
                    print(parameters)
                if velocity > best_velocity:
                    best_velocity = velocity
                    print(f"Up to now, the best_velocity in the generation is {best_velocity}")
                '''

        #Once we get the first generation, we only need to calculate the velocity of children_list because we already know the velocties of parent_list
        else:
            for i in range(len(children_list)):
                print(f"Parent_Children_#{i}")
                parameters = children_list[i]
                num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, spring_rest, spring_ind, center_point\
                    = getMassSpringProperties(arm_length = parameters[:,0], edge = EDGE, long_diagonal = LONG_DIAGONAL, spring_k = SPRING_K)
                different_arm_spring_index = splitSpring(spring_ind, num_mass, num_spring, arm_length = parameters[:,0])
                print(f"The parameters are {parameters}")
                
                mass_pos, center_point, total_energy_list = \
                    runOneGeneration(parameters, num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, spring_rest, spring_ind, different_arm_spring_index, DT, T_MAX, OMEGA)     
                velocity = getFitnessVelocity(mass_pos[14], T_MAX, center_point)

                print(f"the velocity at children {i} is {velocity}")
                
                '''
                #visulize the initial robot shape, close the window to continue
                lsd = o3d.geometry.LineSet()
                lsd.points = o3d.utility.Vector3dVector(mass_pos)
                lsd.lines = o3d.utility.Vector2iVector(spring_ind)
                plane = createPlane()
                #o3d.visualization.draw_geometries([lsd])
                customVisualization([lsd, plane])
                '''
                
                velocity_list.append(velocity)
                '''
                if velocity > 2:
                    print('WARNING！！！！！！')
                    print(parameters)
                if velocity > best_velocity:
                    best_velocity = velocity
                    print(f"Up to now, the best_velocity in the generation is {best_velocity}")
                '''

        
        velocity_list, parent_list = selection(velocity_list, parent_children_list, parent_size)
        best_velocity_list.append(velocity_list[0])
        
        #save the data in case the computer crash
        if generation % 1 == 0:
            with open('evolve_progress_save_parent_velocity_1.pkl', 'wb') as pickle_file:
                pickle.dump(parent_list, pickle_file)
                pickle.dump(best_velocity_list, pickle_file)

    end_time = time.process_time()
    print(f"Total time is {end_time - start_time}")
    return parent_list, best_velocity_list

In [ ]:
#run this if you start from beginning
parent_list, best_velocity_list = evolutionary()

Generation: 0
Parent_Children_#0
The parameters are [[0 0 3327 0 0]
 [7 3 19591 0.27989360991831347 -1.2801946570136793]
 [7 10 11436 -0.270474915745796 0.1253839391246827]
 [5 5 10019 -0.045894909745403445 0.8606907514776583]
 [4 6 8362 -0.31563739970263205 0.9240470181702722]
 [1 10 10921 -0.29872738924537645 0.3687445148715881]
 [4 9 12814 0.2575673952249189 0.0025551606505014313]]
the velocity at parent_children 0 is !!!0.05810868424305292!!!
Parent_Children_#1
The parameters are [[0 0 4832 0 0]
 [3 9 17391 -0.2769733913876099 -0.3118741360594105]
 [6 4 10363 -0.35802060501508715 -0.32255008533033847]
 [2 11 18443 -0.2792607303489648 -0.5068127710901126]
 [1 6 8801 -0.3723941072841088 0.870568942084688]
 [4 3 11165 0.055311512296400855 1.6577129516302995]
 [6 3 11464 0.18769691851597448 -1.8266472782441463]]
the velocity at parent_children 1 is !!!0.015150360723006873!!!
Parent_Children_#2
The parameters are [[0 0 4051 0 0]
 [5 7 10718 0.03921440112579633 0.8794496980070983]
 [1 3 

the velocity at parent_children 18 is !!!0.00865732438314528!!!
Parent_Children_#19
The parameters are [[0 0 3819 0 0]
 [6 5 14670 -0.3776426589802478 -0.15539833764391986]
 [1 5 14722 0.0006876101405940416 -0.5063369493987693]
 [3 8 9732 -0.04245003837658795 0.04199386086765666]
 [7 8 10390 -0.052648474441977566 0.5583535125379402]
 [6 8 10020 -0.08029232363069 0.2361963178978459]
 [2 3 13508 0.2786990076804229 -0.589417286623803]]
the velocity at parent_children 19 is !!!0.016246331924151904!!!
Parent_Children_#20
The parameters are [[0 0 4403 0 0]
 [4 9 10495 0.33900458339303685 -0.26454181220339734]
 [6 9 12415 0.09026317183479282 -0.33720481830049914]
 [1 10 8943 0.05722692936599427 -0.4205820421025279]
 [4 6 17644 0.1939906049868545 -0.5970004423278126]
 [1 5 18902 0.2187827579274244 1.1259381094868104]
 [7 5 15378 0.14993627124359732 -0.6605140615316832]]
the velocity at parent_children 20 is !!!0.047342977499809015!!!
Parent_Children_#21
The parameters are [[0 0 4653 0 0]
 [4 8

the velocity at parent_children 37 is !!!0.08128373597790194!!!
Parent_Children_#38
The parameters are [[0 0 4168 0 0]
 [7 3 12937 -0.1953368740839957 -2.077340548528634]
 [6 11 18539 0.20966757400522307 0.1774185772684047]
 [1 8 17377 0.03238876065876052 0.31913093135485093]
 [2 9 9690 -0.21730185613837896 0.5731723952624722]
 [6 3 11217 -0.15854993861236866 -1.8465498432072132]
 [7 6 15016 0.08771103695801585 -0.03760821492916144]]
the velocity at parent_children 38 is !!!0.010449285375225992!!!
Parent_Children_#39
The parameters are [[0 0 5000 0 0]
 [3 11 19021 -0.2129452907778931 -0.3091837558117564]
 [3 6 16129 0.06966761910628139 -0.3104608222581947]
 [1 8 8985 -0.25949294507659293 0.2557559809236345]
 [7 3 13159 -0.20689798147478147 1.4125205690464928]
 [5 10 13527 -0.1171054402167071 -0.5116110083224824]
 [4 5 14784 -0.2970300796810146 -1.009255695110535]]
the velocity at parent_children 39 is !!!0.0760774569663454!!!
Parent_Children_#40
The parameters are [[0 0 3114 0 0]
 [7 1

the velocity at children 5 is 0.038623163003525314
Parent_Children_#6
The parameters are [[0 0 4653 0 0]
 [6 8 11400 0.21534635659769186 -1.8266472782441463]
 [6 8 12554 0.35344095695516053 0.2757448457859406]
 [7 11 13753 0.300668344432483 0.4154494438772831]
 [7 11 13551 0.12330586166986701 -0.3978794751196174]
 [6 4 7104 0.020074797374790676 0.9886934951292186]
 [6 5 14483 -0.20646538601553682 -0.12457591777630328]]
the velocity at children 6 is 0.023995344155687166
Parent_Children_#7
The parameters are [[0 0 4145 0 0]
 [7 9 13159 -0.3369120902890994 1.3769601643354112]
 [7 11 14666 0.1405589959261817 -0.1356321693177005]
 [7 5 8700 -0.14768105442790658 -0.8660708052261203]
 [6 7 8852 -0.20621693224431745 0.1703487358759499]
 [6 7 19913 0.357456564736946 0.22492538759603167]
 [3 6 18819 -0.05473830718971773 0.4068364721625757]]
the velocity at children 7 is 0.058796656216659846
Parent_Children_#8
The parameters are [[0 0 5000 0 0]
 [3 11 19021 -0.2129452907778931 -0.3091837558117564

the velocity at children 0 is 0.13896063837014394
Parent_Children_#1
The parameters are [[0 0 4269 0 0]
 [4 9 19785 0.3309344505460826 0.621645933144035]
 [1 8 7579 -0.20720307800748783 0.5825441886591443]
 [2 10 7064 -0.015941224847406343 -0.1509008139394764]
 [6 5 17128 0.1330437194016838 -0.4429200059618723]
 [4 4 10969 0.3702846148336745 -0.3182059574499718]
 [2 10 18443 -0.01909335683839719 -0.5068127710901126]]
the velocity at children 1 is 0.1320428237076122
Parent_Children_#2
The parameters are [[0 0 4269 0 0]
 [4 9 19785 0.3309344505460826 0.621645933144035]
 [5 8 7579 -0.14768105442790658 -0.8660708052261203]
 [1 10 8801 -0.180172400147237 0.22818363820963494]
 [6 5 9590 0.1330437194016838 -0.4429200059618723]
 [4 4 10302 0.3702846148336745 -0.3182059574499718]
 [2 8 18443 -0.19243019744410284 -0.5068127710901126]]
the velocity at children 2 is 0.03318349192793961
Parent_Children_#3
The parameters are [[0 0 4269 0 0]
 [4 9 16751 -0.180172400147237 -0.5970004423278126]
 [1 8 7

the velocity at children 20 is 0.09904650032485815
Parent_Children_#21
The parameters are [[0 0 3702 0 0]
 [4 5 7471 -0.13790539212612407 0.05132654500148415]
 [3 10 19102 -0.29920008323923164 0.621645933144035]
 [5 9 9743 -0.20621693224431745 0.1703487358759499]
 [7 10 17723 0.07567043291707948 -0.34505971915607997]
 [5 6 7761 0.222280832341894 1.1412697305930917]
 [5 9 10667 0.07136009191466669 -0.07377551145833006]]
the velocity at children 21 is 0.003952336930946762
Parent_Children_#22
The parameters are [[0 0 3048 0 0]
 [6 6 12140 -0.15854993861236866 -1.8465498432072132]
 [2 6 8362 -0.03199855848163902 -0.16748004473381506]
 [4 10 14227 0.14437273838369422 -0.28125190205615946]
 [6 6 9424 0.14993627124359732 0.8606907514776583]
 [5 11 11462 -0.02662444024784988 0.40824618686460534]
 [6 9 10443 -0.2677156876108101 -0.40861157681160154]]
the velocity at children 22 is 0.03062151486277474
Parent_Children_#23
The parameters are [[0 0 4686 0 0]
 [6 9 8622 -0.04054464724651713 -0.45421

the velocity at children 14 is 0.04047584834224112
Parent_Children_#15
The parameters are [[0 0 3393 0 0]
 [2 7 13508 0.2786990076804229 -0.36573729896828194]
 [1 6 14509 -0.3647764903452612 -0.36743418152070273]
 [0 8 9964 -0.25949294507659293 0.30497837995819604]
 [2 10 16869 0.17554491319491994 -0.5596192033769819]
 [5 9 10667 0.07136009191466669 -0.07377551145833006]
 [9 5 19712 -0.2970300796810146 -0.44320862990121124]]
the velocity at children 15 is 0.004695541216235833
Parent_Children_#16
The parameters are [[0 0 5000 0 0]
 [3 11 19021 -0.2129452907778931 -0.3091837558117564]
 [3 6 16129 0.06966761910628139 -0.3104608222581947]
 [1 8 8985 -0.25949294507659293 0.2557559809236345]
 [4 9 19785 -0.20689798147478147 1.4125205690464928]
 [5 12 13527 -0.1171054402167071 -0.5116110083224824]
 [4 5 14784 -0.2970300796810146 -1.009255695110535]]
the velocity at children 16 is 0.02662980382557425
Parent_Children_#17
The parameters are [[0 0 5000 0 0]
 [3 11 19021 -0.2129452907778931 -0.309

the velocity at children 8 is 0.12519468180778257
Parent_Children_#9
The parameters are [[0 0 4403 0 0]
 [4 9 10495 0.33900458339303685 -0.26454181220339734]
 [6 9 12415 0.09026317183479282 -0.33720481830049914]
 [7 6 8943 0.05722692936599427 0.7004877402320977]
 [4 6 17644 0.1939906049868545 -0.5970004423278126]
 [1 4 13421 0.12330586166986701 1.1259381094868104]
 [4 6 17644 0.1939906049868545 -0.5970004423278126]]
the velocity at children 9 is 0.08818368310931755
Parent_Children_#10
The parameters are [[0 0 4909 0 0]
 [3 8 16009 0.34907672561571923 0.5269071786332882]
 [5 5 9590 -0.35497565920647267 0.19237625255074375]
 [3 7 9079 -0.2690218369327606 0.31595257728534043]
 [7 6 14666 -0.2241043191638327 0.7004877402320977]
 [1 7 14509 -0.3647764903452612 -0.36743418152070273]
 [1 10 8943 0.05722692936599427 -0.4205820421025279]]
the velocity at children 10 is 0.01830302988358029
Parent_Children_#11
The parameters are [[0 0 4909 0 0]
 [3 8 16009 0.34907672561571923 0.5269071786332882]


the velocity at children 2 is 0.10742585738746376
Parent_Children_#3
The parameters are [[0 0 4269 0 0]
 [4 9 20651 0.3309344505460826 0.621645933144035]
 [1 8 7579 -0.22979188976282183 0.5825441886591443]
 [7 5 7064 -0.2970300796810146 -0.44320862990121124]
 [2 10 13208 0.11584980809128531 0.043255668696022966]
 [4 4 10969 0.36456691834497074 -0.3182059574499718]
 [4 4 10714 0.36456691834497074 -0.3182059574499718]]
the velocity at children 3 is 0.13092649833595907
Parent_Children_#4
The parameters are [[0 0 4909 0 0]
 [3 8 10445 0.2518035876303095 0.789292238255412]
 [5 5 9590 -0.35497565920647267 0.19237625255074375]
 [3 7 9079 -0.2690218369327606 0.31595257728534043]
 [7 6 14666 -0.2241043191638327 0.7004877402320977]
 [1 7 14509 -0.3647764903452612 -0.36743418152070273]
 [4 8 19999 -0.32739946063381287 0.645327100237814]]
the velocity at children 4 is 0.1420604366149806
Parent_Children_#5
The parameters are [[0 0 4909 0 0]
 [3 8 16009 0.34907672561571923 0.5269071786332882]
 [5 4 

In [ ]:
#run this if you start from previous parent_list, best_velocity_list(used for fitness figure)
#load the data from the pickle file
with open('evolve_progress_save_parent_velocity_1.pkl', 'rb') as pickle_file:
    parent_list = pickle.load(pickle_file)
    best_velocity_list = pickle.load(pickle_file)
print(parent_list, best_velocity_list)
parent_list, best_velocity_list = evolutionary(previous_parent_list = parent_list, previous_best_velocity_list = best_velocity_list)

In [ ]:
plt.plot(range(len(best_velocity_list)), best_velocity_list)
plt.xlabel('Evolution')
plt.ylabel('Fitness Velocity')
plt.title('The fitness plot: velocity at each evolution')
best_velocity_list

# After Run!
## Save and play. You can change the code below to modify your robot and environment properties. With #Change Here, it is highly recommendedto change and see the visual robot. 

In [ ]:
print(parent_list, best_velocity_list)
with open('evolve_progress_save_parent_velocity_1.pkl', 'wb') as pickle_file:
    pickle.dump(parent_list, pickle_file)
    pickle.dump(best_velocity_list, pickle_file)

## After running the evolutionary algorithm, we get the parameters (arm_length, omega, k, b, c). Using the parameters from parent_list to one single robot animation

In [34]:
#important initial properties
WEIGHT = 0.1
ARM_LENGTH = 4 #change the arm length here （Change Here)
ROBOT_HEIGHT =  0.1 * ARM_LENGTH + 0.1 + 0.04 * (ARM_LENGTH-1) #just above the ground （Change Here)
ROBOT_LENGTH =  0.1 * ARM_LENGTH * 2 + 0.2 #the longest distance between all pairs of masses
SPRING_K = 10000
GROUND_K = 100000
DAMPING = 0.999 #(Change Here)
EDGE = 0.1 
SHORT_DIAGONAL = np.sqrt(2)/10
LONG_DIAGONAL = np.sqrt(3)/10
CENTER_POINT = (0,0,0)
RADIUS = 0.01
DT = 0.0001 #(Change Here)
OMEGA = 9 #change the OMEGA here (Change Here)
GRAVITY = 9.81
MU_S = 1.0 #static friction coefficient of rubber on concrete  #(Change Here)
MU_K = 0.8 #sliding friction coefficient of rubber on concrete   #(Change Here)
T = 0
T_MAX = 3 #change the max time here (Change Here)

#get the best parameters
with open('evolve_progress_save_parent_velocity_1.pkl', 'rb') as pickle_file:
    parent_list = pickle.load(pickle_file)
    best_velocity_list = pickle.load(pickle_file)
best_parameters = parent_list[0]

#initialize the mass properties and spring properties
num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, spring_rest, spring_ind, center_point\
                    = getMassSpringProperties(arm_length = best_parameters[:,0], edge = EDGE, long_diagonal = LONG_DIAGONAL, spring_k = SPRING_K)
different_arm_spring_index = splitSpring(spring_ind, num_mass, num_spring, arm_length = best_parameters[:,0])

#visulization setup
lsd = o3d.geometry.LineSet()
lsd.points = o3d.utility.Vector3dVector(mass_pos)
lsd.lines = o3d.utility.Vector2iVector(spring_ind)
plane = createPlane()

#visulize
mass_pos, center_point = runSimulation(plane,lsd, best_parameters, num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, spring_rest, spring_ind, center_point, different_arm_spring_index, DT, T_MAX, OMEGA)

#calculate the velocity
best_velocity = getFitnessVelocity(mass_pos[14], T_MAX, center_point)
print(f"The best_velocity is: {best_velocity}")
#robot_max_size_per_cycle = getRobotMaxSizePerCycle(best_velocity, OMEGA, ROBOT_LENGTH)
#print(f"The robot maximum size per cycle is: {robot_max_size_per_cycle}")

The best_velocity is: 0.19104354890361655


## After running the evolutionary algorithm, we get the parameters (arm_length, omega, k, b, c). Using multiply parameters from parent_list to get robot zoo animation

In [ ]:
#important initial properties
WEIGHT = 0.1
SPRING_K = 10000
GROUND_K = 100000
DAMPING = 0.999 #(Change Here)
EDGE = 0.1 
SHORT_DIAGONAL = np.sqrt(2)/10
LONG_DIAGONAL = np.sqrt(3)/10
RADIUS = 0.01
DT = 0.0001 #(Change Here)
GRAVITY = 9.81
MU_S = 1.0 #static friction coefficient of rubber on concrete  #(Change Here)
MU_K = 0.8 #sliding friction coefficient of rubber on concrete   #(Change Here)
T = 0
T_MAX = 1 #change the max time here (Change Here)

#get the best parameters
with open('evolve_final_parent_list_and_velocity.pkl', 'rb') as pickle_file:
    parent_list = pickle.load(pickle_file)
    best_velocity_list = pickle.load(pickle_file)
    
#visulization setup
lsd = o3d.geometry.LineSet()
lsd.points = o3d.utility.Vector3dVector(mass_pos)
lsd.lines = o3d.utility.Vector2iVector(spring_ind)
plane = createPlane()

In [ ]:
# This function run the simulation in a visual GUI
def runZooSimulation(plane, lsd, parameters, num_mass, mass_m, mass_pos, mass_vel, mass_acc, num_spring, spring_k, \
                  spring_rest, spring_ind, center_point, different_arm_spring_index, DT, T_MAX, OMEGA):
    GROUND_K = 100000
    DAMPING = 0.999
    #EDGE = 0.1 
    #SHORT_DIAGONAL = np.sqrt(2)/10
    #LONG_DIAGONAL = np.sqrt(3)/10
    #CENTER_POINT = (0,0,0)
    #RADIUS = 0.01
    DT = DT
    OMEGA = parameters[:,1]
    GRAVITY = 9.81
    MU_S = 1.0 #static friction coefficient of rubber on concrete
    MU_K = 0.8 #sliding friction coefficient of rubber on concrete
    T = 0
    T_MAX = T_MAX
    
    ended=False
    def signalEnd(vis):
        nonlocal ended
        ended =True
        
    vis = o3d.visualization.VisualizerWithKeyCallback()
    # ref:https://www.glfw.org/docs/3.3/group__keys.html
    # press key Q or ESC to close the window
    vis.register_key_callback(81, signalEnd)# key Q 
    vis.register_key_callback(256, signalEnd)# key escape
    vis.create_window()
    
    vis.add_geometry(lsd)
    vis.add_geometry(plane)   
    
    # view control
    ctr = vis.get_view_control()
    setView(ctr, camera_pos=(1,1,1), lookat=(0, 0, 0), up=(0,0, 1))
    
    loop_num = 0
    #change the spring_k:
    spring_current_k = np.copy(spring_k)
    for i in range(7):
        spring_current_k[different_arm_spring_index[i]] = parameters[i,2]
    
    while T < T_MAX:
        T += DT
        #set the force to zero every loop
        spring_force = np.zeros((num_spring),dtype=np.float64)
        mass_force = np.zeros((num_mass,3),dtype=np.float64)
        

        #change the spring_rest with t
        spring_current_rest = np.copy(spring_rest)

        for i in range(7):
            if i != 0:
                spring_current_rest[different_arm_spring_index[i]] *= 1 + parameters[i,3] * np.sin(OMEGA[i] * T + parameters[i,4])
                
        #get the spring_force
        spring_vec = mass_pos[spring_ind[:,1]] - mass_pos[spring_ind[:,0]]
        spring_vec_norm = np.linalg.norm(spring_vec,axis = 1)
        spring_unit_vec = spring_vec/ spring_vec_norm[:, np.newaxis]
        spring_force = ((spring_vec_norm - spring_current_rest) * spring_current_k)[:,np.newaxis] * spring_unit_vec
        np.add.at(mass_force, spring_ind[:,0], spring_force)
        np.add.at(mass_force, spring_ind[:,1], -spring_force)

        #get the gravity force
        mass_force[:,2] -= mass_m * GRAVITY

        #get the restoration_force
        mass_pos_below_zero = mass_pos[:,2] <= 0 #give a bool array when the z value of mass_pos is below zero

        #get coulomb friction
        normal_force_below_zero = mass_pos[:,2] < 0

        friction_available_index = np.asarray([i for i, val in enumerate(mass_pos_below_zero & normal_force_below_zero) if val])
        if np.any(friction_available_index):
            horizontal_force = np.sqrt(mass_force[friction_available_index,0]**2 + mass_force[friction_available_index,1]**2)
            normal_force = mass_force[friction_available_index,2]
            static_friction_index = np.array([i for i, val in zip(friction_available_index, horizontal_force < (-normal_force * MU_S)) \
                                                if val], dtype = np.int64)
            slide_friction_index = np.array([i for i, val in zip(friction_available_index, horizontal_force >= (-normal_force * MU_S)) \
                                               if val], dtype = np.int64)
            mass_force[static_friction_index,0] = 0.0
            mass_force[static_friction_index,1] = 0.0
            mass_vel[static_friction_index,0] = 0.0
            mass_vel[static_friction_index,1] = 0.0

            for index in slide_friction_index:
                if mass_force[index,0] < 0:
                    mass_force[index,0] += np.abs(mass_force[index,2]) * MU_K
                elif mass_force[index,0] >= 0:
                    mass_force[index,0] -= np.abs(mass_force[index,2]) * MU_K  
                elif mass_force[index,1] < 0:
                    mass_force[index,1] += np.abs(mass_force[index,2]) * MU_K    
                elif mass_force[index,1] >= 0:
                    mass_force[index,1] -= np.abs(mass_force[index,2]) * MU_K    
                    
        if np.any(mass_pos_below_zero):
            mass_force[mass_pos_below_zero, 2] -= GROUND_K * mass_pos[mass_pos_below_zero,2]

        #update the mass position
        mass_acc = mass_force/mass_m[:,np.newaxis]
        mass_vel += mass_acc * DT
        mass_vel *= DAMPING
        mass_pos += mass_vel * DT
        
        if loop_num % 100 == 0:
            lsd.points = o3d.utility.Vector3dVector(mass_pos)
            #o3d.visualization.draw_geometries([lsd])
            vis.update_geometry(lsd)
            vis.poll_events()
            vis.update_renderer()
            
        loop_num += 1
    return mass_pos, center_point